# It's all wrong!

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [2]:
structures = pd.read_csv('./structures.csv', delimiter=";")
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [3]:
xyz = structures[['x','y','z']].values

In [4]:
ss = structures.groupby('molecule_name').size()
ss = ss.cumsum()
ss

molecule_name
dsgdb9nsd_000001     5
dsgdb9nsd_000002     9
dsgdb9nsd_000003    12
dsgdb9nsd_000004    16
dsgdb9nsd_000005    19
dsgdb9nsd_000007    27
dsgdb9nsd_000008    30
dsgdb9nsd_133884    44
dsgdb9nsd_133885    60
dtype: int64

In [5]:
ssx = np.zeros(len(ss) + 1, 'int')
ssx[1:] = ss
ssx

array([ 0,  5,  9, 12, 16, 19, 27, 30, 44, 60])

In [6]:
molecule_id = 5
print(ss.index[molecule_id])
start_molecule = ssx[molecule_id]
end_molecule = ssx[molecule_id+1]
xyz[start_molecule:end_molecule]

dsgdb9nsd_000007


array([[-0.018704,  1.525582,  0.010433],
       [ 0.002104, -0.003882,  0.001999],
       [ 0.994873,  1.939743,  0.002941],
       [-0.542076,  1.923611, -0.865117],
       [-0.525241,  1.914173,  0.900024],
       [ 0.525487, -0.401908,  0.877544],
       [-1.011477, -0.418034,  0.009508],
       [ 0.508626, -0.39247 , -0.887601]])

We can compare with the information we get from the original pandas dataframe

In [7]:
structures_idx = structures.set_index('molecule_name')

In [8]:
structures_idx.loc['dsgdb9nsd_000001'][['x', 'y', 'z']].values

array([[-1.269800e-02,  1.085804e+00,  8.001000e-03],
       [ 2.150000e-03, -6.031000e-03,  1.976000e-03],
       [ 1.011731e+00,  1.463751e+00,  2.770000e-04],
       [-5.408150e-01,  1.447527e+00, -8.766440e-01],
       [-5.238140e-01,  1.437933e+00,  9.063970e-01]])

Looks good.

We can now rewrite our function using our arrays.


In [17]:
def get_fast_dist_matrix(xyz, ssx, molecule_id):
    start_molecule, end_molecule = ssx[molecule_id], ssx[molecule_id+1]
    locs = xyz[start_molecule:end_molecule]    
    print("locs", locs)
    num_atoms = end_molecule - start_molecule
    print("num_atoms", num_atoms)
    loc_tile = np.tile(locs.T, (num_atoms,1,1))
    print("loc_tile", loc_tile)
    dist_mat = np.sqrt((loc_tile - loc_tile.T)**2).sum(axis=1)
    return dist_mat

Let's check we get the same result with both techniques.  We use methane, molecule_id=0

In [18]:
molecule_id = 0
molecule = ss.index[molecule_id]
print(molecule)
get_fast_dist_matrix(xyz, ssx, molecule_id)

dsgdb9nsd_000001
locs [[-1.269800e-02  1.085804e+00  8.001000e-03]
 [ 2.150000e-03 -6.031000e-03  1.976000e-03]
 [ 1.011731e+00  1.463751e+00  2.770000e-04]
 [-5.408150e-01  1.447527e+00 -8.766440e-01]
 [-5.238140e-01  1.437933e+00  9.063970e-01]]
num_atoms 5
loc_tile [[[-1.269800e-02  2.150000e-03  1.011731e+00 -5.408150e-01 -5.238140e-01]
  [ 1.085804e+00 -6.031000e-03  1.463751e+00  1.447527e+00  1.437933e+00]
  [ 8.001000e-03  1.976000e-03  2.770000e-04 -8.766440e-01  9.063970e-01]]

 [[-1.269800e-02  2.150000e-03  1.011731e+00 -5.408150e-01 -5.238140e-01]
  [ 1.085804e+00 -6.031000e-03  1.463751e+00  1.447527e+00  1.437933e+00]
  [ 8.001000e-03  1.976000e-03  2.770000e-04 -8.766440e-01  9.063970e-01]]

 [[-1.269800e-02  2.150000e-03  1.011731e+00 -5.408150e-01 -5.238140e-01]
  [ 1.085804e+00 -6.031000e-03  1.463751e+00  1.447527e+00  1.437933e+00]
  [ 8.001000e-03  1.976000e-03  2.770000e-04 -8.766440e-01  9.063970e-01]]

 [[-1.269800e-02  2.150000e-03  1.011731e+00 -5.408150e-01 

array([[0.      , 1.112708, 1.4101  , 1.774485, 1.761641],
       [1.112708, 0.      , 2.481062, 2.875143, 2.874349],
       [1.4101  , 2.481062, 0.      , 2.445691, 2.467483],
       [1.774485, 2.875143, 2.445691, 0.      , 1.809636],
       [1.761641, 2.874349, 2.467483, 1.809636, 0.      ]])

In [19]:
e1 = np.array([-1.269800e-02, 1.085804e+00, 8.001000e-03])
e2 = np.array([2.150000e-03, -6.031000e-03, 1.976000e-03])

In [22]:
np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2))

-0.9015418463421857

In [33]:
locs

NameError: name 'locs' is not defined

In [ ]:
loc_tile = np.tile(locs.T, (num_atoms,1,1))
dist_mat = np.sqrt((loc_tile - loc_tile.T)**2).sum(axis=1)
return dist_mat

In [28]:
from math import sqrt
epsilon = 1e-5

In [32]:
def numba_dist_matrix_ssert(xyz, ssx, molecule_id):
    start_molecule, end_molecule = ssx[molecule_id], ssx[molecule_id+1]
    locs = xyz[start_molecule:end_molecule]     
   # return locs
    num_atoms = end_molecule - start_molecule
    dmat = np.zeros((num_atoms, num_atoms))
    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            d = sqrt((locs[i,0] - locs[j,0])**2 + (locs[i,1] - locs[j,1])**2 + (locs[i,2] - locs[j,2])**2)
            dmat[i,j] = d
            dmat[j,i] = d
    assert np.abs(dmat[0,1] - np.linalg.norm(locs[0] - locs[1])) < epsilon
    return dmat

for molecule_id in range(structures.molecule_name.nunique()):
    print(molecule_id)
    print(numba_dist_matrix_ssert(xyz, ssx, molecule_id))
    print("\n")

0
[[0.         1.09195258 1.09195163 1.09194683 1.0919477 ]
 [1.09195258 0.         1.78311968 1.78314749 1.78315661]
 [1.09195163 1.78311968 0.         1.78315808 1.78314847]
 [1.09194683 1.78314749 1.78315808 0.         1.78314786]
 [1.0919477  1.78315661 1.78314847 1.78314786 0.        ]]


1
[[0.         1.01719045 1.01718669 1.01720842]
 [1.01719045 0.         1.61852281 1.61871018]
 [1.01718669 1.61852281 0.         1.61870556]
 [1.01720842 1.61871018 1.61870556 0.        ]]


2
[[0.         0.96210734 0.96210572]
 [0.96210734 0.         1.51335775]
 [0.96210572 1.51335775 0.        ]]


3
[[0.       1.199078 2.261178 1.0621  ]
 [1.199078 0.       1.0621   2.261178]
 [2.261178 1.0621   0.       3.323278]
 [1.0621   2.261178 3.323278 0.      ]]


4
[[0.         1.15174862 1.06659748]
 [1.15174862 0.         2.2183461 ]
 [1.06659748 2.2183461  0.        ]]


5
[[0.         1.52962879 1.09495379 1.09495805 1.09496797 2.18248552
  2.18248534 2.18253648]
 [1.52962879 0.         2.1824

# Okay, that is the distance matrix for molecule_id==0. Let's check which molecule it is.

In [11]:
print(ss.index[molecule_id])

dsgdb9nsd_000001


# As expected, it is dsgdb9nsd_000001. But we know it should be very symmetric with the same distances. Let's calculate it using the structures file.

In [12]:
structures.loc[structures['molecule_name']=='dsgdb9nsd_000001']

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


# Let's calculate the distances from the Carbon atom to the Hydrogens

In [13]:
carbon_x = -0.012698
carbon_y = 1.085804
carbon_z = 0.008001

In [14]:
def distance_from_carbon_to_xyz(xyz):
    return np.sqrt( (carbon_x - xyz[0])**2 + (carbon_y - xyz[1])**2 + (carbon_z - xyz[2])**2)

In [15]:
structures.loc[structures['molecule_name']=='dsgdb9nsd_000001', ['x','y','z']].apply(distance_from_carbon_to_xyz, 1)

0    2.084477e-07
1    1.091953e+00
2    1.091952e+00
3    1.091947e+00
4    1.091948e+00
dtype: float64

# We can see the distances are 0 and 1.09 pretty much. Let's check the distance matrix again.

In [16]:
molecule_id = 0
molecule = ss.index[molecule_id]
print(molecule)
get_fast_dist_matrix(xyz, ssx, molecule_id)

dsgdb9nsd_000001


array([[0.        , 1.1127089 , 1.4101004 , 1.7744841 , 1.76164028],
       [1.1127089 , 0.        , 2.48106245, 2.87514325, 2.87434919],
       [1.4101004 , 2.48106245, 0.        , 2.44569075, 2.46748371],
       [1.7744841 , 2.87514325, 2.44569075, 0.        , 1.80963641],
       [1.76164028, 2.87434919, 2.46748371, 1.80963641, 0.        ]])

# That distance is not there. Therefore the distance computation is wrong.